In [ ]:
import sys
import os
# Ajouter le répertoire principal du projet au chemin de recherche des modules
sys.path.append(os.path.abspath('../'))


In [ ]:
# type: ignore
from tensorflow.keras.applications import ResNet50 
from tensorflow.keras.layers import Input, Flatten, Dense, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import numpy as np
import os
import cv2


In [ ]:
# Définir les dimensions de l'image d'entrée (doit correspondre à ce qu'attend ResNet)
IMG_WIDTH = 224
IMG_HEIGHT = 224
IMG_CHANNELS = 3
INPUT_SHAPE = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
NUM_CLASSES = 6
BATCH_SIZE = 32
EPOCHS = 50  # Tu peux ajuster le nombre d'epochs
LEARNING_RATE = 1e-4


In [ ]:
# Charger le modèle ResNet50 pré-entraîné sur ImageNet, sans la couche supérieure (include_top=False)
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=INPUT_SHAPE))


In [ ]:
# Geler les couches du modèle de base pour l'entraînement initial
for layer in base_model.layers:
    layer.trainable = False

# Ajouter nos propres couches de classification
head_model = base_model.output
head_model = AveragePooling2D(pool_size=(7, 7))(head_model)
head_model = Flatten(name="flatten")(head_model)
head_model = Dense(256, activation='relu')(head_model)
head_model = Dense(NUM_CLASSES, activation='softmax')(head_model)

# Créer le modèle final
model = Model(inputs=base_model.input, outputs=head_model)


In [ ]:
# Compiler le modèle
optimizer = Adam(learning_rate=LEARNING_RATE)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
# Charger et prétraiter tes données (assure-toi de redimensionner les images à 224x224)
def load_data(directory, target_size=(IMG_HEIGHT, IMG_WIDTH)):
    data = []
    labels = []
    for dirname in os.listdir(directory):
        path = os.path.join(directory, dirname)
        if os.path.isdir(path):
            for filename in os.listdir(path):
                img_path = os.path.join(path, filename)
                try:
                    image = cv2.imread(img_path)
                    if image is not None:
                        image = cv2.resize(image, target_size)
                        data.append(image)
                        labels.append(dirname)
                except Exception as e:
                    print(f"Erreur de chargement de l'image {img_path}: {e}")
    return np.array(data), np.array(labels)



In [ ]:
path = "../dataset"
datadir = os.path.join(path, "african_plums")
data, labels = load_data(datadir)
data = data.astype('float32') / 255.0



In [ ]:
le = LabelBinarizer()
labels = le.fit_transform(labels)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)



In [ ]:
# Augmentation de données (tu peux l'adapter)
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)

test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow(X_test, y_test, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
# Callbacks (Early Stopping)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)


In [ ]:
# Entraîner le modèle (seulement la tête de classification au début)
print("[INFO] training head of the network...")
H = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    validation_data=test_generator,
    validation_steps=len(X_test) // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping]
)


In [ ]:
# Fine-tuning (facultatif) - Décongeler certaines couches supérieures du modèle de base
print("[INFO] fine-tuning the last few layers of the network...")
for layer in base_model.layers[-20:]: # Décongeler les 20 dernières couches
    layer.trainable = True

# Ré-compiler le modèle avec un taux d'apprentissage plus faible pour le fine-tuning
optimizer = Adam(learning_rate=LEARNING_RATE / 10)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])



In [ ]:
# Continuer l'entraînement avec les couches dégelées
FINE_TUNE_EPOCHS = EPOCHS // 2 # Entraîner pour moins d'epochs en fine-tuning
H_fine_tune = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // BATCH_SIZE,
    validation_data=test_generator,
    validation_steps=len(X_test) // BATCH_SIZE,
    epochs=FINE_TUNE_EPOCHS,
    callbacks=[early_stopping]
)

# Évaluer le modèle final
print("[INFO] evaluating the network...")
loss, accuracy = model.evaluate(test_generator, steps=len(X_test) // BATCH_SIZE)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")



In [ ]:
# Générer le rapport de classification
# type: ignore
from sklearn.metrics import classification_report, confusion_matrix 
import matplotlib.pyplot as plt
import seaborn as sns

y_pred_probs = model.predict(test_generator, steps=len(X_test) // BATCH_SIZE)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes
class_names = le.classes_

print("[INFO] generating classification report...")
report = classification_report(y_true, y_pred, target_names=class_names)
print(report)

# Afficher la matrice de confusion (facultatif)
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()